In [ ]:
import requests
import json
import pymongo
import pandas as pd

from config import yelpkey

In [ ]:
# Load Kaggle CSV and drop duplicates

kaggle_df = pd.read_csv('Datafiniti_Fast_Food_Restaurants.csv')
kaggle_df = kaggle_df.drop(columns='categories').drop_duplicates()
#test_df = kaggle_df.head(5)
#test_df

In [ ]:
base_url = "https://api.yelp.com/v3/businesses/search"
headers = {'Authorization': f'Bearer {yelpkey}'}

conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.fastfood_db

In [ ]:
row_number = 0
    row_number += 1

for index, row in kaggle_df.iterrows():
    row_number += 1
    params = {
        'term': row['name'],
        'latitude': row['latitude'],
        'longitude': row['longitude'],
        'limit': 1
    }    
    try:
        response = requests.get(base_url, params=params, headers=headers).json()['businesses'][0]
        response['kaggle_id'] = row['id']
        db.yelp.insert_one(response)
        print(f'Processing record {row_number}')
    except KeyError, IndexError:
        print(f'No result for record {row_number}')

In [ ]:
response

In [ ]:
print(yelp_lat)
print(csv_lat)
(yelp_lat - csv_lat) * 110000